In [3]:
import pandas as pd
import datetime as dt
import numpy as np

### We will perform web scraping on the BestPrice website to identify products with price ranges similar to those in our original dataset.

In [2]:
df_original = pd.read_csv(r"C:\Users\Αντώνης Καρούσης\OneDrive\pythoncourses\FinalProjectClean.csv")
df_original

,product_id,product_category_name,month_year,qty,total_price,freight_price,unit_price,product_weight_g,product_score,customers,...,ps3,fp3,lag_price,variance1,variance2,variance3,base_unit_price,base_comp_1,base_comp_2,base_comp_3
0,bed1,bed_bath_table,2017-05-01,1,57.4375,18.875000,57.437500,350,4.0,57,...,4.0,18.875000,45.900000,-48.887653,-78.627907,0.000000,57.4375,112.375,247.977128,57.4375
1,bed1,bed_bath_table,2017-06-01,3,172.3125,16.166667,57.437500,350,4.0,61,...,4.0,16.166667,45.950000,-48.887653,-78.014354,0.000000,57.4375,112.375,247.977128,57.4375
2,bed1,bed_bath_table,2017-07-01,6,344.6250,18.550000,57.437500,350,4.0,123,...,4.0,18.550000,45.950000,-48.887653,-77.585366,0.000000,57.4375,112.375,247.977128,57.4375
3,bed1,bed_bath_table,2017-08-01,4,229.7500,17.859375,57.437500,350,4.0,90,...,4.0,17.859375,45.950000,-48.887653,-76.968550,0.000000,57.4375,112.375,247.977128,57.4375
4,bed1,bed_bath_table,2017-09-01,2,114.8750,18.875000,57.437500,350,4.0,54,...,4.0,18.875000,45.950000,-48.887653,-71.878603,0.000000,57.4375,112.375,247.977128,57.4375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,bed5,bed_bath_table,2017-05-01,1,268.7500,10.950000,268.750000,9750,4.4,57,...,4.0,18.875000,214.950000,139.154616,0.000000,367.899891,265.0000,112.375,265.000000,57.4375
672,bed5,bed_bath_table,2017-06-01,10,2612.5000,26.652500,261.250000,9750,4.4,61,...,4.0,16.166667,215.000000,132.480534,0.000000,354.842220,265.0000,112.375,265.000000,57.4375
673,bed5,bed_bath_table,2017-07-01,59,15118.7500,27.744915,256.250000,9750,4.4,123,...,4.0,18.550000,209.000000,128.031146,0.000000,346.137106,265.0000,112.375,265.000000,57.4375
674,bed5,bed_bath_table,2017-08-01,52,12968.7500,24.266106,249.387255,9750,4.4,90,...,4.0,17.859375,205.000000,121.924142,0.000000,334.188909,265.0000,112.375,265.000000,57.4375


In [3]:
df_original['product_id'].nunique()

52

In [4]:
df_original.groupby('product_category_name')['product_id'].nunique()

product_category_name
bed_bath_table            5
computers_accessories     6
consoles_games            2
cool_stuff                5
furniture_decor           4
garden_tools             10
health_beauty            10
perfumery                 2
watches_gifts             8
Name: product_id, dtype: int64

### The CSV file was created in Power BI. Using our original dataset, we built a table visual in the Report view, as shown below, and then exported it in CSV format.

In [5]:
scraping_df = pd.read_csv(r"C:\Users\Αντώνης Καρούσης\OneDrive\pythoncourses\pricerange.csv")
scraping_df

,product_id,product_category_name,Price Range
0,watches1,watches_gifts,135.00 – 187.67
1,watches2,watches_gifts,135.00 – 187.67
2,watches6,watches_gifts,135.00 – 187.67
3,watches8,watches_gifts,135.00 – 187.67
4,watches1,watches_gifts,187.67 – 255.48
...,...,...,...
92,bed1,bed_bath_table,49.05 – 57.44
93,bed4,bed_bath_table,49.05 – 57.44
94,bed2,bed_bath_table,57.44 – 106.13
95,bed3,bed_bath_table,57.44 – 106.13


In [6]:
scraping_df['product_category_name'].unique()

array(['watches_gifts', 'perfumery', 'health_beauty', 'garden_tools',
       'furniture_decor', 'cool_stuff', 'consoles_games',
       'computers_accessories', 'bed_bath_table'], dtype=object)

In [7]:
import pandas as pd
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

### We are creating a new column with the product category names in a form that can be used inside a url. 

In [8]:
scraping_df['category_only'] = scraping_df['product_category_name'].str.replace('_', '+', regex=True).str.strip()

scraping_df = scraping_df.replace('bed+bath+table', 'bed+table')
scraping_df = scraping_df.replace('watches+gifts', 'men+watches')
scraping_df = scraping_df.replace('perfumery', 'fragrances')
scraping_df = scraping_df.replace('cool+stuff', 'cool')
scraping_df = scraping_df.replace('consoles+games', 'ps4+games')
scraping_df = scraping_df.replace('health+beauty', 'cosmetics')
scraping_df = scraping_df.replace('garden+tools', 'Garden+Multi-Tools')
scraping_df

,product_id,product_category_name,Price Range,category_only
0,watches1,watches_gifts,135.00 – 187.67,men+watches
1,watches2,watches_gifts,135.00 – 187.67,men+watches
2,watches6,watches_gifts,135.00 – 187.67,men+watches
3,watches8,watches_gifts,135.00 – 187.67,men+watches
4,watches1,watches_gifts,187.67 – 255.48,men+watches
...,...,...,...,...
92,bed1,bed_bath_table,49.05 – 57.44,bed+table
93,bed4,bed_bath_table,49.05 – 57.44,bed+table
94,bed2,bed_bath_table,57.44 – 106.13,bed+table
95,bed3,bed_bath_table,57.44 – 106.13,bed+table


In [9]:
scraping_df['category_only'].unique()

array(['men+watches', 'fragrances', 'cosmetics', 'Garden+Multi-Tools',
       'furniture+decor', 'cool', 'ps4+games', 'computers+accessories',
       'bed+table'], dtype=object)

In [10]:
scraping_df[scraping_df['category_only']=='men+watches']['Price Range'].unique()

array(['135.00 – 187.67', '187.67 – 255.48', '255.48 – 436.00',
       '97.28 – 135.00'], dtype=object)

### We are creating a list with all the urls we will be using for the webscraping!

In [11]:

url_list=[]
for cat in list(scraping_df['category_only'].unique()):
    cat_price_ranges_list=list(scraping_df[scraping_df['category_only']==cat]['Price Range'].unique())
    for pr in cat_price_ranges_list:
        if cat=='men+watches':
            min_range=pr[:pr.find('.')]
            s=pr.split(' ')
            max_range=s[2][:s[2].find('.')]
            url=f'https://www.bestprice.gr/cat/848/andrika-rologia.html?qo={cat}&qid=aaFMjcLsEYC&from=search&min={min_range}00&max={max_range}00'
            url_list.append(url)
        elif cat=='ps4+games':
            min_range=pr[:pr.find('.')]
            s=pr.split(' ')
            max_range=s[2][:s[2].find('.')]
            url=f'https://www.bestprice.gr/cat/6041/playstation-4-games.html?qo={cat}&qid=aaFMjcLsEMU&from=search&min={min_range}00&max={max_range}00'
            url_list.append(url)
        else:
            #print(pr)
            
            min_range=pr[:pr.find('.')]
            s=pr.split(' ')
            max_range=s[2][:s[2].find('.')]
            #print(max_range,min_range)
            url=f'https://www.bestprice.gr/search?q={cat}&min={min_range}00&max={max_range}00'
            url_list.append(url)
print(url_list)

['https://www.bestprice.gr/cat/848/andrika-rologia.html?qo=men+watches&qid=aaFMjcLsEYC&from=search&min=13500&max=18700', 'https://www.bestprice.gr/cat/848/andrika-rologia.html?qo=men+watches&qid=aaFMjcLsEYC&from=search&min=18700&max=25500', 'https://www.bestprice.gr/cat/848/andrika-rologia.html?qo=men+watches&qid=aaFMjcLsEYC&from=search&min=25500&max=43600', 'https://www.bestprice.gr/cat/848/andrika-rologia.html?qo=men+watches&qid=aaFMjcLsEYC&from=search&min=9700&max=13500', 'https://www.bestprice.gr/search?q=fragrances&min=14600&max=14700', 'https://www.bestprice.gr/search?q=fragrances&min=14700&max=17800', 'https://www.bestprice.gr/search?q=fragrances&min=6200&max=7100', 'https://www.bestprice.gr/search?q=fragrances&min=7100&max=14600', 'https://www.bestprice.gr/search?q=cosmetics&min=2400&max=6400', 'https://www.bestprice.gr/search?q=cosmetics&min=40200&max=45500', 'https://www.bestprice.gr/search?q=cosmetics&min=6400&max=9900', 'https://www.bestprice.gr/search?q=cosmetics&min=9900&

### We are starting our webscraping process. We will take an url with the product category name and the price range. Then we will choose from the first 5 products,enter each one's website and take the prices from the first 3 competitors. If there aren't any or there are less than 3 competitors we will replace the missing values with the mean of the existing competitors prices and the price of the product we first chose.

In [46]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re
from datetime import datetime

# ---------- Selenium setup ----------
chrome_options = Options()
chrome_options.add_argument("--start-maximized")




# ---------- helpers ----------

def start_driver():
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-infobars")
    return webdriver.Chrome(options=chrome_options)

def parse_euro(text):
    if not text:
        return None
    m = re.search(r"([\d\.,]+)", text)
    if not m:
        return None
    s = m.group(1).replace(" ", "")
    if "." in s and "," in s:
        s = s.replace(".", "").replace(",", ".")
    elif "," in s and "." not in s:
        s = s.replace(",", ".")
    elif s.count(".") > 1 and "," not in s:
        parts = s.split(".")
        s = "".join(parts[:-1]) + "." + parts[-1]
    try:
        return float(s)
    except:
        return None

def accept_cookies(driver):
    try:
        btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable(
                (By.XPATH,
                 "//*[self::button or self::a][contains(.,'Αποδέχομαι') or contains(.,'Accept') or contains(.,'Συμφωνώ')]")
            )
        )
        btn.click()
        time.sleep(1)
        print("✔ Cookies accepted")
    except:
        pass

def scroll_once(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 0.35);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 0.7);")
    time.sleep(1)

def wait_for_listing_products(driver, wait, retries=3):
    for attempt in range(retries):
        try:
            scroll_once(driver)
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h3.p__title a")))
            return True
        except:
            time.sleep(2)
    return False

def get_top3_offer_prices(driver):
    """
    ΜΟΝΟ οι 3 πρώτες τιμές από τα 3 πρώτα offer rows (καταστήματα).
    """
    local_wait = WebDriverWait(driver, 12)
    local_wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.prices__price-wrapper")))

    rows = driver.find_elements(By.CSS_SELECTOR, "div.prices__price-wrapper")

    vals = []
    for row in rows[:3]:
        try:
            price_text = row.find_element(By.CSS_SELECTOR, "a").text
            vals.append(parse_euro(price_text))
        except:
            vals.append(None)

    while len(vals) < 3:
        vals.append(None)

    return vals[0], vals[1], vals[2]

def extract_min_max_from_url(url):
    m1 = re.search(r"min=(\d+)", url)
    m2 = re.search(r"max=(\d+)", url)
    if not (m1 and m2):
        return None, None
    return int(m1.group(1))/100, int(m2.group(1))/100

def extract_category_from_url(url):
    # για /search?q=... ή /cat/... ?qo=...
    m = re.search(r"[?&](?:q|qo)=([^&]+)", url)
    return m.group(1) if m else None



# MAIN LOOP

all_rows = []
scrape_date = datetime.now().strftime("%Y-%m-%d")

BATCH_SIZE = 3   # 🔥 κάθε 3 URLs restart
driver = start_driver()
wait = WebDriverWait(driver, 20)

for idx, url in enumerate(url_list, 1):

     # 🔁 restart driver κάθε BATCH_SIZE URLs
    if idx % BATCH_SIZE == 1 and idx != 1:
        print("🔄 Restarting driver to avoid crash...")
        driver.quit()
        driver = start_driver()
        wait = WebDriverWait(driver, 20)

    
    print(f"\n[{idx}/{len(url_list)}] {url}")

    try:
        driver.get(url)
    except Exception as e:
        print("❌ Driver crashed εδώ, skip URL:", e)
        continue

    
    time.sleep(2)
    accept_cookies(driver)

    ok = wait_for_listing_products(driver, wait, retries=3)
    if not ok:
        print("  ❌ Δεν φόρτωσαν προϊόντα σε listing. Skipping.")
        continue

    name_elems = driver.find_elements(By.CSS_SELECTOR, "h3.p__title a")

    # We are taking the first 5 products of the page
    products = []
    for el in name_elems[:5]:
        name = el.get_attribute("title") or el.text.strip()
        product_url = el.get_attribute("href")

        # card-level discounted_price
        discounted_price = None
        try:
            card = el.find_element(By.XPATH, "./ancestor::div[@data-id][1]")
            price_text = card.find_element(By.CSS_SELECTOR, "div.p__price--current").text
            discounted_price = parse_euro(price_text)
        except:
            discounted_price = None

        products.append({
            "product_name": name,
            "discounted_price": discounted_price,
            "product_url": product_url
        })

    # For each product: go to the product page and get top3 competitors
    for p in products:
        c1 = c2 = c3 = None
        status = "no_product_url"
        err = ""

        if p["product_url"]:
            try:
                driver.get(p["product_url"])
                time.sleep(2.5)
                accept_cookies(driver)
                status = "ok"
                err = ""
            except Exception as e:
                print("   ❌ Crash σε product page, το προσπερνάω:", e)
                c1 = c2 = c3 = None
                status = "crash_product_page"
                err = str(e)[:200]
        
            
            
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 0.75);")
                time.sleep(1)
                c1, c2, c3 = get_top3_offer_prices(driver)
                # if there are not 3 offers:
                if c1 is None and c2 is None and c3 is None:
                    status = "no_offers_found"
            except Exception as e:
                c1, c2, c3 = None, None, None
                status = "error"
                err = str(e)[:200]

        
        mn, mx = extract_min_max_from_url(url)
        cat = extract_category_from_url(url)

        all_rows.append({
            "status": status,
            "error_msg": err,
            "date": scrape_date,
            "category": cat,
            "min_range": mn,
            "max_range": mx,
            "source_url": url,
            "product_name": p["product_name"],
            "discounted_price": p["discounted_price"],
            "competitor_higher_price1": c1,
            "competitor_higher_price2": c2,
            "competitor_higher_price3": c3,
            "product_url": p["product_url"],
        })

# Closing of the driver
driver.quit()


# FINAL DF 

df_full = pd.DataFrame(all_rows)


df_final = df_full[[
    "category",
    "product_name",
    "discounted_price",
    "competitor_higher_price1",
    "competitor_higher_price2",
    "competitor_higher_price3"
]]

print("\nDONE. Rows:", len(df_final))
print(df_final.head(20))

# Save
df_final.to_csv("bestprice_all_urls_top5_top3.csv", index=False, encoding="utf-8-sig")
print("\nSaved: bestprice_all_urls_top5_top3.csv")

df_full[df_full["status"] != "ok"][["category","product_name","product_url","status","error_msg"]].head(30)





[1/35] https://www.bestprice.gr/cat/848/andrika-rologia.html?qo=men+watches&qid=aaFMjcLsEYC&from=search&min=13500&max=18700
✔ Cookies accepted

[2/35] https://www.bestprice.gr/cat/848/andrika-rologia.html?qo=men+watches&qid=aaFMjcLsEYC&from=search&min=18700&max=25500

[3/35] https://www.bestprice.gr/cat/848/andrika-rologia.html?qo=men+watches&qid=aaFMjcLsEYC&from=search&min=25500&max=43600
🔄 Restarting driver to avoid crash...

[4/35] https://www.bestprice.gr/cat/848/andrika-rologia.html?qo=men+watches&qid=aaFMjcLsEYC&from=search&min=9700&max=13500
✔ Cookies accepted

[5/35] https://www.bestprice.gr/search?q=fragrances&min=14600&max=14700

[6/35] https://www.bestprice.gr/search?q=fragrances&min=14700&max=17800
🔄 Restarting driver to avoid crash...

[7/35] https://www.bestprice.gr/search?q=fragrances&min=6200&max=7100
✔ Cookies accepted

[8/35] https://www.bestprice.gr/search?q=fragrances&min=7100&max=14600

[9/35] https://www.bestprice.gr/search?q=cosmetics&min=2400&max=6400
🔄 Restart

,category,product_name,product_url,status,error_msg
65,Garden+Multi-Tools,Karcher MP 145 Multi Power...,https://www.bestprice.gr/t...,error,Message: \n
66,Garden+Multi-Tools,Κάνη MP 145 Multi Power Ka...,https://www.bestprice.gr/t...,error,Message: \n
67,Garden+Multi-Tools,Τσεκούρι Πολυεργαλείο Robe...,https://www.bestprice.gr/t...,error,Message: \n
68,Garden+Multi-Tools,Karcher MP 145 κάνη Multi ...,https://www.bestprice.gr/t...,error,Message: \n
69,Garden+Multi-Tools,Camillus CAMTRAX(TM) Multi...,https://www.bestprice.gr/t...,error,Message: \n
71,Garden+Multi-Tools,Τσεκούρι Robens Multi Axe,https://www.bestprice.gr/t...,error,Message: \n
72,Garden+Multi-Tools,Τσεκούρι Robens Multi Axe,https://www.bestprice.gr/t...,error,Message: \n
73,Garden+Multi-Tools,Τσεκούρι Robens Multi Axe,https://www.bestprice.gr/t...,error,Message: \n
74,Garden+Multi-Tools,Κάννη Ψεκασμού MJ 180 Mult...,https://www.bestprice.gr/t...,error,Message: \n
78,Garden+Multi-Tools,WOLF-GARDEN Κοντάρι μπαταρ...,https://www.bestprice.gr/t...,error,Message: \n


In [4]:
df=pd.read_csv(r"C:\Users\Αντώνης Καρούσης\OneDrive\pythoncourses\bestprice_all_urls_top5_top3.csv")
df

,category,product_name,discounted_price,competitor_higher_price1,competitor_higher_price2,competitor_higher_price3
0,men+watches,Casio Edifice Ανδρικό Ρολόι Χρονογράφος Solar ...,163.88,220.00,163.88,174.80
1,men+watches,Casio Edifice Ανδρικό Ρολόι Χρονογράφος Solar ...,163.88,220.00,163.88,178.20
2,men+watches,Casio G-Shock Ανδρικό Ρολόι Χρονογράφος με Μαύ...,145.00,220.00,145.00,169.00
3,men+watches,Casio Edifice Ανδρικό Ρολόι Χρονογράφος με Αση...,160.00,172.79,160.00,163.00
4,men+watches,Casio Edifice Ανδρικό Ρολόι Χρονογράφος με Αση...,138.00,155.00,138.00,156.08
...,...,...,...,...,...,...
170,bed+table,N4 Παιδικό Κρεβάτι Κουκέτα Μονό για Στρώμα 90x...,61.78,61.78,437.00,NaN
171,bed+table,Artekko Chinoise Handmade Flower Bed Side Tabl...,94.12,NaN,NaN,NaN
172,bed+table,Artekko Chinoise Handmade Flower Bed Side Tabl...,87.78,NaN,NaN,NaN
173,bed+table,VEVOR Overbed Table Mobile Bed Side Desk with ...,104.26,NaN,NaN,NaN


In [7]:
df[df['category']=='furniture+decor'].sort_values('discounted_price',ascending=False)

,category,product_name,discounted_price,competitor_higher_price1,competitor_higher_price2,competitor_higher_price3
82,furniture+decor,Κομοδίνο DKD Home Decor Έλατο Μέταλλο (48 x 40...,126.60,NaN,NaN,NaN
80,furniture+decor,JK Home Decoration Καρέκλα Κουζίνας 52472,116.82,116.82,118.80,NaN
83,furniture+decor,Βοηθητικό Τραπέζι DKD Home Decor Καθρέφτης Χρυ...,115.70,NaN,NaN,NaN
84,furniture+decor,Foot-of-bed Bench DKD Home Decor Μαύρο Μέταλλο...,115.42,NaN,NaN,NaN
81,furniture+decor,Βοηθητικό Τραπεζάκι DKD Home Decor Καφέ 60 x 6...,110.30,NaN,NaN,NaN
97,furniture+decor,DKD Home Decor Καρέκλα Τραπεζαρίας S3010311,96.90,96.90,NaN,NaN
98,furniture+decor,Gift Decor Κεφαλάρι Κρεβατιού Τοίχου Ξύλινο Μα...,80.00,80.00,NaN,NaN
95,furniture+decor,Elmark Kubo 97KUBORGB/N,75.50,75.50,96.63,102.32
96,furniture+decor,Πολυθρόνα S3609681,50.61,50.61,294.80,NaN
99,furniture+decor,"Ολόσωμος Καθρέπτης Δαπέδου 3 x 38,5 x135 εκατο...",48.79,NaN,NaN,NaN


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   product_name              175 non-null    object 
 1   discounted_price          175 non-null    float64
 2   competitor_higher_price1  140 non-null    float64
 3   competitor_higher_price2  137 non-null    float64
 4   competitor_higher_price3  121 non-null    float64
dtypes: float64(4), object(1)
memory usage: 7.0+ KB


In [13]:
df_nan_rows = df[df.isna().any(axis=1)]
df_nan_rows

,category,product_name,discounted_price,competitor_higher_price1,competitor_higher_price2,competitor_higher_price3
65,Garden+Multi-Tools,Karcher MP 145 Multi Power Jet Κάνη Ρύθμισης Π...,66.40,NaN,NaN,NaN
66,Garden+Multi-Tools,Κάνη MP 145 Multi Power Karcher (2.643-239.0),63.70,NaN,NaN,NaN
67,Garden+Multi-Tools,Τσεκούρι Πολυεργαλείο Robens Multi Axe,66.95,NaN,NaN,NaN
68,Garden+Multi-Tools,Karcher MP 145 κάνη Multi Power Jet,64.99,NaN,NaN,NaN
69,Garden+Multi-Tools,Camillus CAMTRAX(TM) Multi Purpose Hatchet wit...,66.98,NaN,NaN,NaN
71,Garden+Multi-Tools,Τσεκούρι Robens Multi Axe,69.00,NaN,NaN,NaN
72,Garden+Multi-Tools,Τσεκούρι Robens Multi Axe,69.00,NaN,NaN,NaN
73,Garden+Multi-Tools,Τσεκούρι Robens Multi Axe,69.00,NaN,NaN,NaN
74,Garden+Multi-Tools,Κάννη Ψεκασμού MJ 180 Multi Jet 3 σε 1 για Κ 7...,69.00,NaN,NaN,NaN
78,Garden+Multi-Tools,WOLF-GARDEN Κοντάρι μπαταρίας BS 140 eM e-mult...,115.00,NaN,NaN,NaN


In [16]:
price_cols = [
    "discounted_price",
    "competitor_higher_price1",
    "competitor_higher_price2",
    "competitor_higher_price3"
]

# 1) row-wise mean of all values (ignores NaN)
row_mean = df[price_cols].mean(axis=1, skipna=True)

# 2) If a row has ALL NaN (rare), set global mean of discounted_price
global_mean_discount = df["discounted_price"].mean(skipna=True)
row_mean = row_mean.fillna(global_mean_discount)

# 3) Fill all NaN values with the row_mean of the row
df[price_cols] = df[price_cols].apply(
    lambda col: col.fillna(row_mean)
)



df

,category,product_name,discounted_price,competitor_higher_price1,competitor_higher_price2,competitor_higher_price3
0,men+watches,Casio Edifice Ανδρικό Ρολόι Χρονογράφος Solar ...,163.88,220.00,163.88,174.800000
1,men+watches,Casio Edifice Ανδρικό Ρολόι Χρονογράφος Solar ...,163.88,220.00,163.88,178.200000
2,men+watches,Casio G-Shock Ανδρικό Ρολόι Χρονογράφος με Μαύ...,145.00,220.00,145.00,169.000000
3,men+watches,Casio Edifice Ανδρικό Ρολόι Χρονογράφος με Αση...,160.00,172.79,160.00,163.000000
4,men+watches,Casio Edifice Ανδρικό Ρολόι Χρονογράφος με Αση...,138.00,155.00,138.00,156.080000
...,...,...,...,...,...,...
170,bed+table,N4 Παιδικό Κρεβάτι Κουκέτα Μονό για Στρώμα 90x...,61.78,61.78,437.00,186.853333
171,bed+table,Artekko Chinoise Handmade Flower Bed Side Tabl...,94.12,94.12,94.12,94.120000
172,bed+table,Artekko Chinoise Handmade Flower Bed Side Tabl...,87.78,87.78,87.78,87.780000
173,bed+table,VEVOR Overbed Table Mobile Bed Side Desk with ...,104.26,104.26,104.26,104.260000


In [17]:
df.to_csv("bestprice_all_urls_top5_top3.csv", index=False, encoding="utf-8-sig")